In [1]:
import numpy as np
import pypsa

In [12]:
# Crear instancia de tipo network y añadir tres buses (nodos)
network = pypsa.Network()

n_buses = 3

for i in range(n_buses):
    network.add("Bus", f"Bus {i}", v_nom=20.0)

# Mostramos los buses creados
network.buses

,v_nom,type,x,y,carrier,unit,location,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,,
Bus 0,20.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,
Bus 1,20.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,
Bus 2,20.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,


In [13]:
# Ahora creamos tres líneas en forma de anillo definiendo los
# parámetros que queramos para cada una

# Linea 1 ---> Buses 1 y 2
# Linea 2 ---> Buses 2 y 3
# Linea 3 ---> Buses 3 y 1
for i in range(n_buses):
    network.add(
        "Line",
        f"Line {i}",
        bus0=f"Bus {i}",
        bus1=f"Bus {(i+1) % n_buses}",
        x=0.1,
        r=0.01
    )

network.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
Line 0,Bus 0,Bus 1,,0.1,0.01,0.0,0.0,0.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Line 1,Bus 1,Bus 2,,0.1,0.01,0.0,0.0,0.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Line 2,Bus 2,Bus 0,,0.1,0.01,0.0,0.0,0.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Podemos añadir un generador en el bus 0
network.add("Generator", "Gen", bus="Bus 0", p_set=100, control="PQ")

network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Gen,Bus 0,PQ,,0.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [15]:
# Añadimos una carga en uno de los buses
network.add("Load", "My Load", bus="Bus 1", p_set=100)

network.loads

,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
My Load,Bus 1,,,100.0,0.0,-1.0,True


In [16]:
# Se pueden asignar valores a parámetros de carga y otros elementos
# después de haberlos definido
network.loads.qset = 100.0

In [17]:
# Podemos resolver el flujo de potencia mediante el método de
# Newton-Raphson para un sistema de ecuaciones no lineales
network.pf()

INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x00000166223A2490> for snapshots Index(['now'], dtype='object', name='snapshot')


{'n_iter': SubNetwork  0
 snapshot     
 now         3,
 'error': SubNetwork             0
 snapshot                
 now         2.004701e-12,
 'converged': SubNetwork     0
 snapshot        
 now         True}

In [18]:
# Ya se tiene el resultado, con lo que se puede consultar
# la potencia activa resultante en las líneas
network.lines_t.p0

,Line 0,Line 1,Line 2
snapshot,,,
now,66.778181,-33.333333,-33.361212


In [20]:
# También el ángulo de los voltajes en los buses
network.buses_t.v_ang * 180 / np.pi

Bus,Bus 0,Bus 1,Bus 2
snapshot,,,
now,0.0,-0.956705,-0.477919


In [21]:
# O la magnitud de dichos voltajes
network.buses_t.v_mag_pu

Bus,Bus 0,Bus 1,Bus 2
snapshot,,,
now,1.0,0.998191,0.999061


In [ ]:
# Lo que obtenemos es el estado estacionario al que convergería
# la red electrica "network" que hemos creado en la situación
# definida